In [2]:
import sys
import os
import time

import numpy as np
from tqdm.notebook import tqdm

# add parent dir to be able to import utils file
parent_dir = os.path.abspath('../../')
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from duqling_r_interface import DuqlingRInterface
from duqling import Duqling as DuqlingPyInterface
duq_r  = DuqlingRInterface()
duq_py = DuqlingPyInterface()

In [3]:
def lhs_array(n: int, d: int, seed: int | None = None) -> np.ndarray:
    rng = np.random.default_rng(seed)
    u = rng.random((n, d))
    perms = np.column_stack([rng.permutation(n) for _ in range(d)])
    return (perms + u) / n

In [4]:
NUM_SAMPLES = 100

stochastic_piston_kwargs = dict( 
    Ta_generate = lambda:0.5, 
    P0_generate = lambda:0.5 
)

speedup = []

for fname in tqdm(duq_r.quack().fname):
    kwargs = stochastic_piston_kwargs if fname == 'stochastic_piston' else dict()
    func_info_r  = duq_r .quack(fname)
    func_info_py = duq_py.quack(fname)

    # not all functions in the R duqling repo have 'stochastic' as a quack key
    func_info_r  = {k:v for k,v in func_info_r .items() if k!='stochastic'}
    func_info_py = {k:v for k,v in func_info_py.items() if k!='stochastic'}

    try:
        assert func_info_r.keys() == func_info_py.keys()
    except:
        print(f'{fname} has inconsistent quack keys across language implemetations')

    for k,v_r in func_info_r.items():
        v_py = func_info_py[k]

        try: 
            assert type(v_r) == type(v_py)
        except AssertionError:
            print(f'{fname} has inconsistent {k} types across language implementations')

        try:
            if isinstance(v_r, np.ndarray):
                assert (np.isclose(v_r, v_py)).all()
            else:
                assert v_r == v_py
        except AssertionError:
            print(f'{fname} has inconsistent {k} values across language implementations')

    input_dim = func_info_r['input_dim']
    X = lhs_array(NUM_SAMPLES, input_dim)
    try:
        t0 = time.time()
        y_r = np.column_stack([duq_r .duq(x=x, f=fname, **kwargs) for x in X])
        t_r = time.time() - t0
        t0 = time.time()
        y_p = np.column_stack([duq_py.duq(x=x, f=fname, **kwargs) for x in X])
        t_p = time.time() - t0

        speedup.append((t_r, t_p))
    except:
        print(f'An error occured while testing {fname}')

    try:
        assert np.isclose(y_r, y_p).all()  # output array shapes should match and values should be more or less identical
    except AssertionError:
        print(f'{fname} has inconsistent function outputs across language implementations')

  0%|          | 0/61 [00:00<?, ?it/s]

ocean_circ has inconsistent function outputs across language implementations


R[write to console]: Error in if (N[t] <= 0) { : missing value where TRUE/FALSE needed

R[write to console]: In addition: 
R[write to console]: Warning messages:

R[write to console]: 1: 
R[write to console]: In stats::rbinom(3, c(S[t - 1], I[t - 1], R[t - 1]), x[6:8]) :
R[write to console]: 
 
R[write to console]:  NAs produced

R[write to console]: 2: 
R[write to console]: In stats::rbinom(1, R[t - 1] - deaths[3], x[9]) :
R[write to console]:  NAs produced



borehole_low_fidelity has inconsistent input_range values across language implementations
An error occured while testing dts_sirs


## Summary of issues

- `ocean_circ` is stochastic, so the failed equivalence test isn't totally surprising
- `dts_sirs` fails to terminate early when N[t] is null ('N[t] <= 0' breaks in this case)
- `ignition` uses different equation from paper
    - The paper uses $r^5 \bigl[1 + 100000 \bigl(1 + \operatorname{erf}(10 (r − 2))\bigr)\bigr]$
    but the code uses $r^5 \bigl[1+100000 \bigl(2 \operatorname{cdf}(10\sqrt{2} (r-2))\bigr)\bigr]$
- `borehole_low_fidelity` uses the upper bound 80 instead of 820 as the documentation and library describe